In [20]:
import requests
import random
import numpy as np
from collections import Counter

class JogoDeForca:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code == 200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro:", r.status_code)

    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        self.estado_palavra = ["_"] * len(self.palavra)
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                indices = [idx for idx, char in enumerate(self.palavra) if char == letra]
                for idx in indices:
                    self.estado_palavra[idx] = letra
                return indices
            else:
                self.vidas -= 1
                return []
        return False

    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra == palavra:
                return True
            else:
                self.vidas = 0
                return False


In [21]:
class Jogador:
    def __init__(self, vocabulario):
        self.vocabulario = vocabulario

    def calcular_entropia(self, palavras, letras_tentadas):
        contagem_letras = Counter("".join(palavras))
        total_letras = sum(contagem_letras.values())
        
        entropias = {letra: - (contagem_letras[letra] / total_letras) * np.log2(contagem_letras[letra] / total_letras) 
                     for letra in contagem_letras if letra not in letras_tentadas}
        return entropias

    def escolher_letra(self, estado_palavra, letras_tentadas):
        palavras_compatíveis = [palavra for palavra in self.vocabulario if len(palavra) == len(estado_palavra)]
        palavras_compatíveis = [p for p in palavras_compatíveis if all(
            estado_palavra[i] == "_" or estado_palavra[i] == p[i] for i in range(len(estado_palavra)))]

        entropias = self.calcular_entropia(palavras_compatíveis, letras_tentadas)
        if entropias:
            return max(entropias, key=entropias.get)
        return None


In [ ]:
def simular_jogos(jogador, num_jogos=100):
    resultados = []
    jogo = JogoDeForca()
    
    for i in range(1, num_jogos + 1):
        num_letras = jogo.novo_jogo()
        letras_tentadas = set()
        venceu = False
        
        while jogo.vidas > 0 and "_" in jogo.estado_palavra:
            letra = jogador.escolher_letra(jogo.estado_palavra, letras_tentadas)
            if letra is None:
                break
            letras_tentadas.add(letra)
            jogo.tentar_letra(letra)
                
            if "_" not in jogo.estado_palavra:
                venceu = True
                break

        resultados.append(venceu)
        status = "Vitória" if venceu else "Derrota"
        print(f"Jogo {i}: {status}")
        print(f"Palavra: {jogo.palavra}")
        print(f"Vidas restantes: {jogo.vidas}\n{'-'*30}")

    total_vitorias = sum(resultados)
    print(f"\nTotal de jogos: {num_jogos}")
    print(f"Total de vitórias: {total_vitorias}")
    print(f"Taxa de vitória: {total_vitorias / num_jogos * 100:.2f}%")

# Inicialização do jogador e simulação
jogo = JogoDeForca()
jogador = Jogador(jogo.content)
simular_jogos(jogador, num_jogos=100)


Jogo 1: Vitória
Palavra: apimentares
Vidas restantes: 4
------------------------------
Jogo 2: Vitória
Palavra: abiscoitaste
Vidas restantes: 4
------------------------------
Jogo 3: Derrota
Palavra: bicais
Vidas restantes: 0
------------------------------
Jogo 4: Vitória
Palavra: penitenciei
Vidas restantes: 1
------------------------------
Jogo 5: Vitória
Palavra: olvidaras
Vidas restantes: 2
------------------------------
Jogo 6: Vitória
Palavra: embocai
Vidas restantes: 2
------------------------------
Jogo 7: Vitória
Palavra: despertador
Vidas restantes: 5
------------------------------
Jogo 8: Vitória
Palavra: equilibrasses
Vidas restantes: 4
------------------------------
Jogo 9: Vitória
Palavra: engatilharias
Vidas restantes: 3
------------------------------
Jogo 10: Vitória
Palavra: avermelhamos
Vidas restantes: 5
------------------------------

Total de jogos: 10
Total de vitórias: 9
Taxa de vitória: 90.00%
